In [2]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import matplotlib as plt
import numpy as np

In [3]:
data = pd.read_excel('/Users/user/Library/CloudStorage/OneDrive-TheUniversityofManchester/PDRA/Sequencing/stoke_imapact_data/Manchester_WithMeans_250612.xlsx')

In [ ]:
list(data.columns)

In [ ]:
col_to_remove = ['Date_of_Enrollment',
'Time_of_symptom_onset 🔒',
'Time_symptoms_were_recognized 🔒',
'Time_last_known_well_(LKW) 🔒',
'Date/Time_Blood_Drawn_',
'blooddrawdiff',
'MoCA_V2',
'MoCA_V3',
'MoCA_V4',
'MoCA_V5',
'6mo_imaging',
'30mo_imaging',
'Qualifying_Stroke_Date',
'V3_Date',
'V4_Date',
'V5_Date',
'V3_Post_Stroke_Days',
'V4_Post_Stroke_Days',
'V5_Post_Stroke_Days',
'TimePoint.x',
'I_feel_fatigued.x',
'I_feel_weak_all_over.x',
"I_feel_listless_('washed_out').x",
'I_feel_tired.x',
'I_have_trouble_starting_things_because_I_am_tired.x',
'I_have_trouble_finishing_things_because_I_am_tired.x',
'I_have_energy.x',
'I_am_able_to_do_my_usual_activities.x',
'I_need_to_sleep_during_the_day.x',
'I_am_too_tired_to_eat.x',
'I_need_help_doing_my_usual_activities.x',
'I_am_frustrated_by_being_too_tired_to_do_the_things_I_want_to_do.x',
'I_have_to_limit_my_social_activity_because_I_am_tired.x',
'FACIT_Fatigue_Scale_Total_(prorated,_if_answers_missing)_.x',
'Are_you_basically_satisfied_with_your_life?.x',
'Have_you_dropped_many_of_your_activities_and_interests?.x',
'Do_you_feel_that_your_life_is_empty?.x',
'Do_you_often_get_bored?.x',
'Are_you_in_good_spirits_most_of_the_time?.x',
'Are_you_afraid_that_something_bad_is_going_to_happen_to_you?.x',
'Do_you_feel_happy_most_of_the_time?.x',
'Do_you_often_feel_helpless?.x',
'Do_you_prefer_to_stay_at_home,_rather_than_going_out_and_doing_things?.x',
'Do_you_feel_that_you_have_more_problems_with_memory_than_most?.x',
'Do_you_think_it_is_wonderful_to_be_alive_now?.x',
'Do_you_feel_worthless_the_way_you_are_now?.x',
'Do_you_feel_full_of_energy?.x',
'Do_you_feel_that_your_situation_is_hopeless?.x',
'Do_you_think_that_most_people_are_better_off_than_you_are?.x',
'TimePoint.y',
'I_feel_fatigued.y',
'I_feel_weak_all_over.y',
"I_feel_listless_('washed_out').y",
'I_feel_tired.y',
'I_have_trouble_starting_things_because_I_am_tired.y',
'I_have_trouble_finishing_things_because_I_am_tired.y',
'I_have_energy.y',
'I_am_able_to_do_my_usual_activities.y',
'I_need_to_sleep_during_the_day.y',
'I_am_too_tired_to_eat.y',
'I_need_help_doing_my_usual_activities.y',
'I_am_frustrated_by_being_too_tired_to_do_the_things_I_want_to_do.y',
'I_have_to_limit_my_social_activity_because_I_am_tired.y',
'FACIT_Fatigue_Scale_Total_(prorated,_if_answers_missing)_.y',
'Are_you_basically_satisfied_with_your_life?.y',
'Have_you_dropped_many_of_your_activities_and_interests?.y',
'Do_you_feel_that_your_life_is_empty?.y',
'Do_you_often_get_bored?.y',
'Are_you_in_good_spirits_most_of_the_time?.y',
'Are_you_afraid_that_something_bad_is_going_to_happen_to_you?.y',
'Do_you_feel_happy_most_of_the_time?.y',
'Do_you_often_feel_helpless?.y',
'Do_you_prefer_to_stay_at_home,_rather_than_going_out_and_doing_things?.y',
'Do_you_feel_that_you_have_more_problems_with_memory_than_most?.y',
'Do_you_think_it_is_wonderful_to_be_alive_now?.y',
'Do_you_feel_worthless_the_way_you_are_now?.y',
'Do_you_feel_full_of_energy?.y',
'Do_you_feel_that_your_situation_is_hopeless?.y',
'Do_you_think_that_most_people_are_better_off_than_you_are?.y',
'TimePoint',
'I_feel_fatigued',
'I_feel_weak_all_over',
"I_feel_listless_('washed_out')",
'I_feel_tired',
'I_have_trouble_starting_things_because_I_am_tired',
'I_have_trouble_finishing_things_because_I_am_tired',
'I_have_energy',
'I_am_able_to_do_my_usual_activities',
'I_need_to_sleep_during_the_day',
'I_am_too_tired_to_eat',
'I_need_help_doing_my_usual_activities',
'I_am_frustrated_by_being_too_tired_to_do_the_things_I_want_to_do',
'I_have_to_limit_my_social_activity_because_I_am_tired',
'FACIT_Fatigue_Scale_Total_(prorated,_if_answers_missing)_',
'Are_you_basically_satisfied_with_your_life?',
'Have_you_dropped_many_of_your_activities_and_interests?',
'Do_you_feel_that_your_life_is_empty?',
'Do_you_often_get_bored?',
'Are_you_in_good_spirits_most_of_the_time?',
'Are_you_afraid_that_something_bad_is_going_to_happen_to_you?',
'Do_you_feel_happy_most_of_the_time?',
'Do_you_often_feel_helpless?',
'Do_you_prefer_to_stay_at_home,_rather_than_going_out_and_doing_things?',
'Do_you_feel_that_you_have_more_problems_with_memory_than_most?',
'Do_you_think_it_is_wonderful_to_be_alive_now?',
'Do_you_feel_worthless_the_way_you_are_now?',
'Do_you_feel_full_of_energy?',
'Do_you_feel_that_your_situation_is_hopeless?',
'Do_you_think_that_most_people_are_better_off_than_you_are?',
'Normed_MoCA_V3',
'Normed_MoCA_V4',
'Normed_MoCA_V2']


data = data.drop(columns=col_to_remove)

In [6]:
print(data['Normed_MoCA_V5'])
list(data.dtypes)

0      27.0
1      27.0
2       NaN
3      26.0
4       NaN
       ... 
191    24.8
192    26.0
193     4.0
194    30.0
195    10.0
Name: Normed_MoCA_V5, Length: 196, dtype: float64


[dtype('O'),
 dtype('O'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('float64'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('

In [7]:
categorical_cols = data.select_dtypes(include=['object']).columns
le = LabelEncoder()
for col in categorical_cols:
    data[col] = le.fit_transform(data[col].astype(str)) 

In [8]:
print("Category Mapping:", le.classes_)

Category Mapping: ['THROAT INFECTION' 'nan' 'osteomyelitis']


In [9]:
data = data.dropna(subset = 'Normed_MoCA_V5')

In [15]:
X = data.drop(columns='Normed_MoCA_V5')
y = data['Normed_MoCA_V5']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)

In [ ]:
bst = XGBRegressor(
    n_estimators=200,
    max_depth=2,        
    learning_rate=1,
    colsample_bytree=0.8,
    reg_alpha=0.001,
    gamma=0.1,
    random_state=8,    
)

# Train 
bst.fit(X_train, y_train, 
        eval_set=[(X_test, y_test)], 
        verbose=False)

# predictions
preds = bst.predict(X_test)

importance = bst.feature_importances_
feature_names = X_train.columns

feature_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance
}).sort_values('Importance', ascending=False)

mse = mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)
mae = np.mean(np.abs(y_test - preds))

print(f"Mean Squared Error: {mse:.2f}")
print(f"R² Score: {r2:.3f}")
print(f"Mean Absolute Error: {mae:.2f}")

In [86]:
print("Top 10 Most Important Features:")
print(feature_importance.head(10))

# And see which patients were predicted
results = pd.DataFrame({
    'Row_Index': X_test.index,
    'Actual_MoCA': y_test.values,
    'Predicted_MoCA': preds,
    'Error': y_test.values - preds
})
print(results.head())

Top 10 Most Important Features:
                       Feature  Importance
44  White_Cell_Count_(109/l) 🔒    0.255139
21                ProcSpeed_V3    0.151714
16                         HDL    0.118544
38                  MemFact_V5    0.080195
18                       HbA1C    0.077321
0                     Study_ID    0.075820
34              Impaired_at_V4    0.071112
72      no_med_classes_taken 🔒    0.045248
40               GlobalTest_V5    0.039435
32               GlobalTest_V4    0.022342
   Row_Index  Actual_MoCA  Predicted_MoCA     Error
0         98         19.0       23.634516 -4.634516
1        195         10.0        9.283818  0.716182
2         82         19.0       24.127195 -5.127195
3         55         27.0       26.317574  0.682426
4        194         30.0       27.362335  2.637665
